# Server GPU setting

Ubuntu Workstation

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# FlowNet batch 생성

import (tensorflow, numpy) package <br>
numpy use to make numpy array batch data for training <br>
tensorflow use to make FlowNet structure

In [ ]:
import tensorflow as tf
import numpy as np

dataset : Scene Flow Datasets <br>
model : FlowNet structure <br>

In [ ]:
from model import FlowNet
from dataset import Scene_Flow_disparity

Download Scene Flow Datasets <br>
Make data_paths list for loading data images

In [ ]:
dataset = Scene_Flow_disparity()

Check data method in mode = left, groundTruth mode = disparity

In [ ]:
print(len(dataset.data_paths))
print()
iteration = 0
for dir_ in dataset.data_paths:
    iteration += 1
    if iteration == 200:
        break
    if dataset.data(dir_) is not None:
        print(dir_)
    else:
        print('no left or png path')

# FlowNet Simple training<br>

make FlowNet simple structure <br>
set training environment(input shape, learning rate, FlowNet structure mode)

In [ ]:
import time
import datetime

model = FlowNet(
    img_height = 540, 
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net = model.inference('simple')

start=time.clock()

## training

make training batch input
- train_left_x_batch numpy array
- train_right_x_batch numpy array
<br>

make training batch output
- train_y_batch numpy array
training
- data split in iteration
- no data augmentation

In [ ]:
train_left_x = np.array([])
train_left_x_batch = np.array([])
train_right_x = np.array([])
train_right_x_batch = np.array([])
train_y = np.array([])
train_y_batch = np.array([])

iteration = 0
for dir_ in dataset.data_paths:
    if dataset.data(dir_) is not None:
        left_image, right_image, ground_truth = dataset.data(dir_, 
                                                             (model.model_in_height, model.model_in_width), 
                                                             (model.model_out_height, model.model_out_width))
        
        if train_left_x.size == 0:
            train_left_x = left_image[np.newaxis,:]
            train_right_x = right_image[np.newaxis,:]
            train_y = ground_truth[np.newaxis,:]
        else:
            train_left_x_batch = np.append(train_left_x, left_image[np.newaxis,:], axis = 0)
            train_right_x_batch = np.append(train_right_x, right_image[np.newaxis,:], axis = 0)
            train_y_batch = np.append(train_y, ground_truth[np.newaxis,:], axis = 0)
        
        iteration += 1
        
    elif iteration >= 100:
        hist = net.fit([train_left_x_batch, train_right_x_batch], 
                           train_y_batch, 
                           batch_size=8, 
                           epochs=100, 
                           validation_split=0.2)
        net.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')
        train_left_x = np.array([])
        train_left_x_batch = np.array([])
        train_right_x = np.array([])
        train_right_x_batch = np.array([])
        train_y = np.array([])
        train_y_batch = np.array([])
        iteration = 0

training time check
- day
- hours
- minute
- sec

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

save trained model weight

In [ ]:
net.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')

# FlowNet Correlation training<br>

make FlowNet simple structure <br>
set training environment(input shape, learning rate, FlowNet structure mode)

In [ ]:
import time
import datetime

model = FlowNet(
    img_height = 540, 
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net = model.inference('correlation')

start=time.clock()

## training

make training batch input
- train_left_x_batch numpy array
- train_right_x_batch numpy array
<br>

make training batch output
- train_y_batch numpy array
training
- data split in iteration
- no data augmentation

In [ ]:
train_left_x = np.array([])
train_left_x_batch = np.array([])
train_right_x = np.array([])
train_right_x_batch = np.array([])
train_y = np.array([])
train_y_batch = np.array([])

iteration = 0
for dir_ in dataset.data_paths:
    if dataset.data(dir_) is not None:
        left_image, right_image, ground_truth = dataset.data(dir_, 
                                                             (model.model_in_height, model.model_in_width), 
                                                             (model.model_out_height, model.model_out_width))
        
        if train_left_x.size == 0:
            train_left_x = left_image[np.newaxis,:]
            train_right_x = right_image[np.newaxis,:]
            train_y = ground_truth[np.newaxis,:]
        else:
            train_left_x_batch = np.append(train_left_x, left_image[np.newaxis,:], axis = 0)
            train_right_x_batch = np.append(train_right_x, right_image[np.newaxis,:], axis = 0)
            train_y_batch = np.append(train_y, ground_truth[np.newaxis,:], axis = 0)
        
        iteration += 1
        
    elif iteration >= 100:
        hist = net.fit([train_left_x_batch, train_right_x_batch], 
                           train_y_batch, 
                           batch_size=8, 
                           epochs=100, 
                           validation_split=0.2)
        net.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')
        train_left_x = np.array([])
        train_left_x_batch = np.array([])
        train_right_x = np.array([])
        train_right_x_batch = np.array([])
        train_y = np.array([])
        train_y_batch = np.array([])
        iteration = 0

training time check
- day
- hours
- minute
- sec

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

save trained model weight

In [ ]:
net.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')